# SetUP

In [58]:
setwd("/lustre/user/liclab/jialm/Project/SCOPEC/analysis/stripes/")

In [59]:
suppressPackageStartupMessages({
library(dplyr)
library(tidyverse)
library(data.table)
library(ggplot2)
library(edgeR)
library(RColorBrewer)
library(GenomicRanges)
library(ComplexHeatmap)
#library(igraph)
})

# adjust

In [ ]:
stripes.25 <- read.table("./25kb/RG.eN.merged.stripes")
stripes.25 <- stripes.25 %>% 
mutate(V5=case_when((V3-V2==25000) & (V5 >= V2) ~ V5,
                    (V3-V2==25000) & (V5 < V2) ~ V2,
                    (V3-V2 >25000 ~ V5)),
       V3=case_when((V6-V5==25000) & (V3 > V6) ~ V6,
                    (V6-V5==25000) & (V3 <= V6) ~ V3,
                    (V6-V5 >25000 ~ V3))
       ) %>% distinct()
write.table(stripes.25,"./25kb/RG.eN.merged.stripes.adjust",col.names = F,row.names = F,sep="\t",quote=F)

In [21]:
stripes.50 <- read.table("./50kb/RG.eN.merged.stripes.50kb")
stripes.50 <- stripes.50 %>% 
mutate(V5=case_when((V3-V2==50000) & (V5 >= V2) ~ V5,
                    (V3-V2==50000) & (V5 < V2) ~ V2,
                    (V3-V2 >50000 ~ V5)),
       V3=case_when((V6-V5==50000) & (V3 > V6) ~ V6,
                    (V6-V5==50000) & (V3 <= V6) ~ V3,
                    (V6-V5 >50000 ~ V3))
       ) %>% distinct()
write.table(stripes.50,"./50kb/RG.eN.merged.stripes.50kb.adjust",col.names = F,row.names = F,sep="\t",quote=F)

# 25kb

In [60]:
setwd("/lustre/user/liclab/jialm/Project/SCOPEC/analysis/stripes/25kb.res/")

In [61]:
file.ls <- list.files("./",pattern="sta")

In [62]:
stripes.sta <- lapply(file.ls,read.table)

In [63]:
names(stripes.sta) <- gsub(".allValidPairs.cis.bedpe.sta","",file.ls) 

In [64]:
Input <- cbind(stripes.sta$RG.eN.merged.stripe.V1.RG.allV1,stripes.sta$RG.eN.merged.stripes.PFC.RG.all[,7],
      stripes.sta$RG.eN.merged.stripe.V1.eN.allV1[,7],stripes.sta$RG.eN.merged.stripes.PFC.eN.all[,7]) %>%
set_names("seq1","start1","end1","seq2","start2","end2","RG.1","RG.2","eN.1","eN.2")

In [65]:
data <- Input[,-(1:6)] 
group <- c(rep("RG",2),rep("eN",2))
genes <- Input[,1:6]
dge <- DGEList(counts = data, genes = genes, group = group)
#keep <- filterByExpr(dge, group = group)
dge <- calcNormFactors(dge)
#dge <- dge[keep,]
dge <- estimateDisp(dge)
design <- model.matrix(~ 0 + group, data = dge$samples)
fit <- glmQLFit(dge, design = design)
AvsB <- makeContrasts(groupeN-groupRG, levels=design)
qlf <- glmQLFTest(fit, contrast= AvsB)


Using classic mode.



In [14]:
RGtoeN <- topTags(qlf, n = Inf)$table %>% mutate(Group=case_when(logFC >=0.4 & FDR<=0.05~"Up",
                                                       logFC <= -0.4 & FDR<=0.05~"Down",
                                                        abs(logFC) <0.4 | FDR>0.05~"Not"))
RGtoeN %>% group_by(Group) %>% summarise(n=n())

Group,n
<chr>,<int>
Down,195
Not,1998
Up,227


In [15]:
write.table(RGtoeN,"./RG2eN.25kb.stripes.edgeR.res.txt",col.names = F,row.names = F,sep="\t",quote=F)

# 50kb

In [30]:
setwd("/lustre/user/liclab/jialm/Project/SCOPEC/analysis/stripes/50kb/")

In [31]:
file.ls <- list.files("./",pattern="sta")

In [32]:
file.ls

[1] "RG.eN.merged.stripes.50kb.PFC.eN.all.allValidPairs.cis.bedpe.sta" 
[2] "RG.eN.merged.stripes.50kb.PFC.RG.all.allValidPairs.cis.bedpe.sta" 
[3] "RG.eN.merged.stripes.50kb.V1.eN.allV1.allValidPairs.cis.bedpe.sta"
[4] "RG.eN.merged.stripes.50kb.V1.RG.allV1.allValidPairs.cis.bedpe.sta"

In [33]:
stripes.sta <- lapply(file.ls,read.table)

In [34]:
names(stripes.sta) <- gsub(".allValidPairs.cis.bedpe.sta","",file.ls) 

In [35]:
Input <- cbind(stripes.sta$RG.eN.merged.stripes.50kb.PFC.RG.all,stripes.sta$RG.eN.merged.stripes.50kb.V1.RG.allV1[,7],
               stripes.sta$RG.eN.merged.stripes.50kb.PFC.eN.all[,7],stripes.sta$RG.eN.merged.stripes.50kb.V1.eN.allV1[,7]) %>%
set_names("seq1","start1","end1","seq2","start2","end2","RG.1","RG.2","eN.1","eN.2")

In [36]:
data <- Input[,-(1:6)] 
group <- c(rep("RG",2),rep("eN",2))
genes <- Input[,1:6]
dge <- DGEList(counts = data, genes = genes, group = group)
#keep <- filterByExpr(dge, group = group)
dge <- calcNormFactors(dge)
#dge <- dge[keep,]
dge <- estimateDisp(dge)
design <- model.matrix(~ 0 + group, data = dge$samples)
fit <- glmQLFit(dge, design = design)
AvsB <- makeContrasts(groupeN-groupRG, levels=design)
qlf <- glmQLFTest(fit, contrast= AvsB)


Using classic mode.



In [37]:
RGtoeN <- topTags(qlf, n = Inf)$table %>% mutate(Group=case_when(logFC >=0.5 & FDR<=0.05~"Up",
                                                       logFC <= -0.5 & FDR<=0.05~"Down",
                                                        abs(logFC) <0.5 | FDR>0.05~"Not"))
RGtoeN %>% group_by(Group) %>% summarise(n=n())

Group,n
<chr>,<int>
Down,119
Not,2836
Up,331


In [38]:
write.table(RGtoeN,"./RG2eN.50kb.stripes.edgeR.res.txt",col.names = F,row.names = F,sep="\t",quote=F)

# HRM

In [66]:
setwd("/lustre/user/liclab/jialm/Project/SCOPEC/analysis/stripes/25_50kb.stripes.liftOver")

## 25kb

In [ ]:
H2R <- read.table("./RG2eN.25kb.stripes.edgeR.res.txt.range.rheMac") %>% 
filter(V5==1) %>% 
mutate(ID=paste0(V1,":",V2+1,"-",V3))
R2M <- read.table("./RG2eN.25kb.stripes.edgeR.res.txt.range.rheMac.mm") %>% 
filter(V5==1) %>% 
mutate(ID=paste0(V1,":",V2+1,"-",V3))

In [53]:
HA <- read.table("./RG2eN.25kb.stripes.edgeR.res.txt") %>% 
mutate(V2=V2+1,H=paste0(V1,":",V2,"-",V6))

In [32]:
HRM <- dplyr::inner_join(H2R,R2M,by=c("ID"="V4"))

Warning message in dplyr::inner_join(H2R, R2M, by = c(ID = "V4")):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 735 of `x` matches multiple rows in `y`.
ℹ Row 991 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship = "many-to-many"` to silence this warning.”


In [40]:
HRM <- HRM[,c(4,6,11)] %>% distinct()
colnames(HRM) <- c("H","R","M")

In [56]:
HRM <- HRM %>% dplyr::left_join(.,HA,by="H") %>%
select(H,R,M,V12)

In [57]:
write.table(HRM, "./RG.eN.25kb.stripes.HRM.paired",col.names = F,row.names = F,sep="\t",quote=F)

In [68]:
HRM <- read.table("./RG.eN.25kb.stripes.HRM.paired.bed")

In [84]:
HA <- read.table("./RG2eN.25kb.stripes.edgeR.res.txt") %>% 
mutate(V2=V2+1)%>% 
mutate(g=case_when(V3-V2==24999 ~ "L",
                   V6-V5==25000 ~ "R")) %>%
select(V1,V2,V6,g)

In [87]:
HRM <- HRM %>% dplyr::left_join(.,HA,by=c("V1"="V1","V2"="V2","V3"="V6"))

In [88]:
write.table(HRM, "./RG.eN.25kb.stripes.HRM.paired.bed",col.names = F,row.names = F,sep="\t",quote=F)

## 50kb

In [ ]:
H2R <- read.table("./RG2eN.50kb.stripes.edgeR.res.txt.range.rheMac") %>% 
filter(V5==1) %>% 
mutate(ID=paste0(V1,":",V2+1,"-",V3))
R2M <- read.table("./RG2eN.50kb.stripes.edgeR.res.txt.range.rheMac.mm") %>% 
filter(V5==1) %>% 
mutate(ID=paste0(V1,":",V2+1,"-",V3))

In [30]:
HRM <- dplyr::inner_join(H2R,R2M,by=c("ID"="V4"))

Warning message in dplyr::inner_join(H2R, R2M, by = c(ID = "V4")):
“Each row in `x` is expected to match at most 1 row in `y`.
ℹ Row 86 of `x` matches multiple rows.
ℹ If multiple matches are expected, set `multiple = "all"` to silence this warning.”


In [31]:
HRM <- HRM[,c(4,6,11)] %>% distinct()
colnames(HRM) <- c("H","R","M")

In [33]:
write.table(HRM, "./RG.eN.50kb.stripes.HRM.paired",col.names = F,row.names = F,sep="\t",quote=F)

In [56]:
range <- read.table("./RG2eN.50kb.stripes.edgeR.res.txt.range") %>% mutate(V2=V2+1)

In [57]:
res <- read.table("./RG.eN.50kb.stripes.HRM.paired.bed") %>%
dplyr::left_join(.,range,by=c("V1","V2","V3")) 

In [59]:
write.table(res[,-11], "./RG.eN.50kb.stripes.HRM.paired",col.names = F,row.names = F,sep="\t",quote=F)